<a href="https://colab.research.google.com/github/MirkoThulke/Python_GoogleCollabs_DividendScreener/blob/main/Mirko_Divididende.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pickle import NONE
import os
import sys
!pip install PyPDF2
import PyPDF2
import pandas as pd
import re
import string
from operator import itemgetter
import time

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

#A)
# Aktien Dividenden sowie Fonds Ausschuettungen :
# COMDIRECT postbox nach "Konto/Depotmitteilung" filtern
# Benoetigte Dokumente : "Dividendengutschrift_", "Ertragsgutschrift_"
# zu entfernende Dokumente : "Ankündigung_Rückzahlung_", "Einlösung_", "Zinsgutschriften_"

#B)
# ZERO Kupon Bonds und Kupon Anleihen, die bis zur Faelligkeit gehalten wurden  :
# COMDIRECT postbox nach "Steuermitteilung" filtern
# Steuermitteilung der eingeloessten BUNDANLEIHEN kopieren

#C) NICHT UNTERSTUETZT
# Aktien-/Anleihen-Veraeusserungsgewinne :
# Dafuer in postbox nach "Ordermitteilungen" Dokumenten filtern .
# Benoetigte Dokumente : "Wertpapierabrechnung Verkauf"

#D)  NICHT UNTERSTUETZT
# Kupon Anleihen die noch gehalten werden
# COMDIRECT postbox nach "Konto/Depotmitteilung" filtern
# Benoetigte Dokumente : "Zinsgutschriften_"



Input_Folder  = '/content/drive/MyDrive/Dividenden_Script/Input_PDF'
Output_Folder = '/content/drive/MyDrive/Dividenden_Script/Output_Excel/'


timestr = time.strftime("%Y%m%d-%H%M%S")
text_dump     = ''
text          = ''
filename      = ''
#list_items = ['Depotnr.:','Depotbestand','Wertpapier-Bezeichnung','WKN/ISIN','per','Emissionsland:', 'STK','Bruttobetrag:', 'Quellensteuer', 'Dividende', 'Ausschüttung', 'Zinsgutschrift']
myList_all = []
myList_processed = []
Konto         = ''

# Find item in list
def search_list(plist, pstring):
  python_indices  = [index for (index, item) in enumerate(plist) if item == pstring]
  if python_indices:
    return int(python_indices[0])
  else:
    print(f"'{pstring}' not found in text.")
    sys.exit(1)  # Exit with a non-zero status (indicates error)
    #return -1  # or None or raise a custom error


# Find first occurance of item in list of lists
def search_list_of_lists(plist, pstring):
  for i in range (0, len(plist)): # lines
      for j in range (0, len(plist[0])): # colums
        if plist[i][j]==pstring:
          found=j
          return j

# convert_string2float
def convert_string2float(pstring):
  temp  = pstring.replace('.','')  # remove 1000 '.'
  temp2  = temp.replace(',','.')  # Replace german ',' to Python float '.'
  temp3 = float(temp2) # Convert to float
  return temp3



# Extract data from dump file
def search_text(pfilename, ptext):
    myList1 = []
    myList2 = []
    string1 = ''

    #split string into list based on spaces
    myList1 = ptext.split()


    # Pruefe ob es sich um Anleihe handelt
    x = ptext.rfind('Zinsen')
    if x > 0:
        ZinsenStaatsanleihe_b = True
    else:
        ZinsenStaatsanleihe_b = False

    # Pruefe ob es sich um eine Zero Kupon Anleihe handelt, die bis zur Faelligkeit gehalten wurde
    x = ptext.rfind('Behandlung: Einlösung')
    if x > 0:
        AnleiheEinloesung_b = True
    else:
        AnleiheEinloesung_b = False

    x = ptext.rfind('Dividende')
    if x > 0:
        Dividende_b = True
    else:
        Dividende_b = False

    x = ptext.rfind('Devisenkurs:')
    if x > 0:
        Devisenkurs_b = True
    else:
        Devisenkurs_b = False

    x = ptext.rfind('Ausschüttung')
    if x > 0:
        Ausschüttung_b = True
    else:
        Ausschüttung_b = False

    x = ptext.rfind('Ertragsgutschrift')
    if x > 0:
        Ertragsgutschrift_b = True
    else:
        Ertragsgutschrift_b = False

    x = ptext.rfind('Quellensteuer')
    if x > 0:
        Quellensteuer_b = True
    else:
        Quellensteuer_b = False


    # remove not required items
    #myList1.remove("abc")





    # Aktie oder Fond
    if Dividende_b == True or Ausschüttung_b == True or Ertragsgutschrift_b == True  :
        print("Aktie / Fond : Dividende / Ausschuettung")

        # remove first element from list : '/SP010000/SP0...'
        myList1.pop(0)
        #print(myList1)

        # Depotnummer
        index1 = search_list(myList1, 'Depotnr.:')
        myList2.append(myList1[index1])
        myList2.append(myList1[index1+1])

        # 'Inhaber'
        string1 = ''
        index1 = search_list(myList1, 'Thulke')
        for x in range(2, index1+1):
          string1 = string1 + ' ' + myList1[x]
        myList2.append('Inhaber')
        myList2.append(string1)

        # Datum
        myList2.append(myList1[0])

        # Typ
        myList2.append('Aktie/Fonds')

        # 'Wertpapier-Bezeichnung'
        string1 = ''
        index1 = search_list(myList1, 'Wertpapier-Bezeichnung')
        index2 = search_list(myList1, 'STK')
        myList2.append(myList1[index1])
        for x in range(index1+5, index2):
          string1 = string1 + ' ' + myList1[x]
        myList2.append(string1)

        # 'WKN/ISIN'
        index1 = search_list(myList1, 'WKN/ISIN')
        index2 = search_list(myList1, 'STK')
        myList2.append(myList1[index1])
        myList2.append(myList1[index2+2])
        myList2.append(myList1[index2])
        myList2.append(myList1[index2+1])

        # 'Emissionsland:'
        string1 = ''
        index1 = search_list(myList1, 'Emissionsland:')
        index2 = search_list(myList1, 'pro')
        myList2.append(myList1[index1])
        for x in range(index1+1, index2-3):
          string1 = string1 + ' ' + myList1[x]
        myList2.append(string1)

        # 'Devisenkurs:'
        if Devisenkurs_b == True:
          index1 = search_list(myList1, 'Devisenkurs:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # convert string to float
        else:
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')

        # 'Dividende' / 'Ausschüttung' / 'Ertragsgutschrift'
        if Dividende_b == True :

          index1 = search_list(myList1, 'Dividende')
          myList2.append(myList1[index1])
          index1 = search_list(myList1, 'Bruttobetrag:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # Convert to float

        elif Ausschüttung_b == True :

          index1 = search_list(myList1, 'Ausschüttung')
          myList2.append(myList1[index1])
          index1 = search_list(myList1, 'Bruttobetrag:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # Convert to float

        elif Ertragsgutschrift_b == True :

          index1 = search_list(myList1, 'Ertragsgutschrift')
          myList2.append(myList1[index1])
          index1 = search_list(myList1, 'Bruttobetrag:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # Convert to float

        else:

          myList2.append('Error')
          index1 = search_list(myList1, 'Error')
          myList2.append('Error')
          myList2.append('Error')
          myList2.append('Error')
          print('ERROR : Weder Dividenden noch Ausschuettung gefunden')


        if Quellensteuer_b == True :

          # 'Quellensteuer'
          index1 = search_list(myList1, 'Quellensteuer')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1-1])
          myList2.append(convert_string2float(myList1[index1-2])) # convert string to float
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # convert string to float

        else:

          # 'Quellensteuer'
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')

        print(len(myList2))

    # Kupon Staatsanleihe, die noch gehalten wird ##################
    elif ZinsenStaatsanleihe_b == True :
        print("Zinsen Staatsanleihe, die noch gehalten wird")
        print(myList1)

        # Depotnummer
        index1 = search_list(myList1, 'Depotnummer:')
        myList2.append('Depotnr.:')
        myList2.append(myList1[index1+1])

        # 'Inhaber'
        index1 = search_list(myList1, 'Thulke')
        string1 = " " + myList1[index1-3]+ " " + myList1[index1-2]+ " " + myList1[index1-1]+ " " + myList1[index1]
        myList2.append('Inhaber')
        myList2.append(string1)

        # Datum
        index1 = search_list(myList1, 'Datum:')
        myList2.append(myList1[index1+1])

        # Typ
        myList2.append('Verzinzte Anleihe')

        # 'Wertpapier-Bezeichnung'
        index1 = search_list(myList1, 'WKN')
        myList2.append('Wertpapier-Bezeichnung')
        myList2.append(myList1[index1-2])

        # 'WKN/ISIN'
        index1 = search_list(myList1, 'WKN')
        myList2.append('WKN/ISIN')
        myList2.append(myList1[index1+5])
        myList2.append('STK')
        myList2.append('n.a.')

        # 'Emissionsland:'
        myList2.append('Emissionsland:')
        myList2.append('Deutschland')

        # 'Devisenkurs:'
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')

        # 'Dividende' / 'Ausschüttung'
        myList2.append('Kupon:')
        index1 = search_list(myList1, 'Gunsten')
        myList2.append('Bruttobetrag:')
        myList2.append(myList1[index1+3])
        myList2.append(convert_string2float(myList1[index1+4])) # Convert to float

        # 'Quellensteuer'
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')


        print(len(myList2))

    # ZERO KUPON Staatsanleihe gehalten bis Faelligkeit ##################
    elif AnleiheEinloesung_b == True :
        print("ZERO KUPON Anleihe bis Faelligkeit gehalten")
        print(myList1)

        # Depotnummer
        index1 = search_list(myList1, 'Depotnummer:')
        myList2.append('Depotnr.:')
        myList2.append(myList1[index1+1])

        # 'Inhaber'
        index1 = search_list(myList1, 'Thulke')
        string1 = " " + myList1[index1-3]+ " " + myList1[index1-2]+ " " + myList1[index1-1]+ " " + myList1[index1]
        myList2.append('Inhaber')
        myList2.append(string1)

        # Datum
        index1 = search_list(myList1, 'Datum:')
        myList2.append(myList1[index1+1])

        # Typ
        myList2.append('Zero Kupon Anleihe')

        # 'Wertpapier-Bezeichnung'
        index1 = search_list(myList1, 'WKN')
        myList2.append('Wertpapier-Bezeichnung')
        myList2.append(myList1[index1-2])

        # 'WKN/ISIN'
        index1 = search_list(myList1, 'WKN')
        myList2.append('WKN/ISIN')
        myList2.append(myList1[index1+5])
        myList2.append('STK')
        myList2.append('n.a.')

        # 'Emissionsland:'
        myList2.append('Emissionsland:')
        myList2.append('Deutschland')

        # 'Devisenkurs:'
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')

        # 'Dividende' / 'Ausschüttung'
        myList2.append('Einloesung:')
        index1 = search_list(myList1, 'Gunsten')
        myList2.append('Bruttobetrag:')
        myList2.append(myList1[index1+3])
        auszahlung = convert_string2float(myList1[index1+4])

        index1 = search_list(myList1, 'Stichtag')
        einzahlung = convert_string2float(myList1[index1+2])
        ertrag = auszahlung + einzahlung # Convert to float
        myList2.append(ertrag)

        # 'Quellensteuer'
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')



        print(len(myList2))

    # Undefiniertes Dokument
    else:
      print("Undefiniertes Dokument")
      return # exit and do not append

    #write result into overall list of lists
    myList_all.append(myList2)

#End of function

for filename in os.listdir(Input_Folder):
    if filename.endswith('.pdf'):
       # Open the PDF file
        with open(os.path.join(Input_Folder, filename), 'rb') as pdf_file:
          print('\n'+filename+'\n')

          # Read the PDF file
          pdf_reader = PyPDF2.PdfReader(pdf_file)

          i = 0
          page = 0
          page_text = str()
          text = str()

          # Extract the text of all pages
          for i, page in enumerate(pdf_reader.pages):

            page_text = page.extract_text()

            if page_text:  # Avoid appending None
              text += page_text + "\n"  # Add new line between pages

          # Print the text
          #print(text)

          # extract data and write into global list : myList_all
          search_text(filename, text)

          # Close the PDF file
          pdf_file.close()
else :
    print('PDFs gelesen ...')




# Create a DataFrame
df1 = pd.DataFrame(myList_all)

# processing
index1 = search_list_of_lists(myList_all, 'Emissionsland:')

print(myList_all)

myList_processed = sorted(myList_all, key=itemgetter(index1+1))


# Create a DataFrame
df2 = pd.DataFrame(myList_processed)

# Excel export
Konto = myList_processed[0][1] # Konto nummer
excel_datei_name = Konto+'_'+time.strftime("%Y%m%d-%H%M%S")+'.xlsx'
excel_datei = Output_Folder+'/'+excel_datei_name

print(excel_datei)
with pd.ExcelWriter(excel_datei) as writer:
    df1.to_excel(writer, sheet_name='Raw_Data', index=False)
    df2.to_excel(writer, sheet_name='Processed_Data', index=False)